In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [22]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import display, Markdown
from typing import List
from enum import Enum

from pydantic import BaseModel, Field
import json
from typing import List

In [23]:
#model = init_chat_model("ministral-8b-latest", model_provider="mistralai", temperature=0)
model = init_chat_model("mistral-large-latest", model_provider="mistralai", temperature=0)

In [24]:
class Period(str, Enum):
    """Enum for periods of life."""
    CHILDHOOD = "childhood"
    ADOLESCENCE = "adolescence"
    YOUNG_ADULTHOOD = "young_adulthood"
    ADULTHOOD = "adulthood"
    OLD_AGE = "old_age"

class Person(BaseModel):
    id: int = Field(description="The unique identifier for the person")
    name: str = Field(description="The name of the person")
    relationship: str = Field(description="The relationship of the person to the user")
    age: int = Field(description="The age of the person")
    occupation: str = Field(description="The occupation of the person")
    hobbies: List[str] = Field(default_factory=list, description="The hobbies of the person")
    description: str = Field(description="A description of the person")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

class Memory(BaseModel):
    people: List[int] = Field(min_items=1, decription="A list of IDs of people involved in the memory")
    place: List[str] = Field(min_items=1, default_factory=list, description="A list of places involved in the memory")
    period: List[str] = Field(min_items=1, default_factory=list)
    emotion: List[str] = Field(min_items=1, default_factory=list)
    description: str = Field(description="A description of the memory")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)
    
class LifeSummary(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")
    occupation: str = Field(description="The occupation of the user")
    acquaintances: List[Person] = Field(min_items=5, default_factory=list, description="A list of acquaintances of the user (friend, colleague,  family)")
    memories: List[Memory] = Field(default_factory=list)

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

# Génération des souvenirs pour la vie

In [25]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage("Tu est un chatbot qui génère des données de synthèse sur les souvenirs sur toute la vie d'une personne agée."),
        ("user", """Invente cinq souvenirs. pour une personne dont voici la description :
            Nom : {name}
            Age : {age}
            Profession : {occupation} 
         Rédige les souvenirs à la première personne,
         en utilisant le vocabulaire et les émotions d'une personne âgée.
         Chaque souvenir doit être spécifique et détaillé, en incluant la référence (id) des personnes impliquées, les lieux, la période de la vie et les émotions ressenties."""),
    ]
)
chain = prompt_template | model.with_structured_output(LifeSummary)

In [ ]:
life_summary = chain.invoke({"name": "Monique", "age": "75", "occupation": "Ancienne Assistante sociale"})

In [10]:
life_summary.model_dump()

{'name': 'Monique',
 'age': 75,
 'occupation': 'Ancienne Assistante sociale',
 'acquaintances': [{'id': 1,
   'name': 'Jean',
   'relationship': 'Mari',
   'age': 78,
   'occupation': 'Retraité',
   'hobbies': ['Jardinage', 'Lecture'],
   'description': 'Mon mari depuis 50 ans, un homme patient et aimant.'},
  {'id': 2,
   'name': 'Sophie',
   'relationship': 'Fille',
   'age': 45,
   'occupation': 'Professeur',
   'hobbies': ['Peinture', 'Randonnée'],
   'description': 'Ma fille aînée, une femme indépendante et passionnée.'},
  {'id': 3,
   'name': 'Pierre',
   'relationship': 'Fils',
   'age': 42,
   'occupation': 'Ingénieur',
   'hobbies': ['Football', 'Cuisine'],
   'description': 'Mon fils cadet, un homme dévoué et toujours prêt à aider.'},
  {'id': 4,
   'name': 'Marie',
   'relationship': "Amie d'enfance",
   'age': 75,
   'occupation': 'Retraitée',
   'hobbies': ['Tricot', 'Voyages'],
   'description': "Mon amie d'enfance, toujours là pour moi dans les bons et les mauvais momen

# Génération de souvenirs pour le mariage

In [9]:
class Person(BaseModel):
    id: int = Field(description="The unique identifier for the person")
    name: str = Field(description="The name of the person")
    relationship: str = Field(description="The relationship of the person to the user")
    age: int = Field(description="The age of the person")
    occupation: str = Field(description="The occupation of the person")
    hobbies: List[str] = Field(default_factory=list, description="The hobbies of the person")
    description: str = Field(description="A description of the person")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

class Memory(BaseModel):
    person: Person = Field(decription="The writer of the memory")
    place: List[str] = Field(min_items=1, default_factory=list, description="A list of places involved in the memory")
    emotion: List[str] = Field(min_items=1, default_factory=list)
    description: str = Field(description="A description of the memory")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)
    
class LifeSummary(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")
    occupation: str = Field(description="The occupation of the user")
    memories: List[Memory] = Field(default_factory=list)

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

In [10]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage("""Tu est un chatbot qui génère des données de synthèse. Par rapport à un évènement, avec différentes perspectives de différentes personnes."""),
        ("user", """L'évènement est le mariage de Thibault, le petit fils de Monique.
        Voici la description de Monique :
            Nom : {name}
            Age : {age}
            Profession : {occupation} 
        Invente 15 souvenirs écrits par des proches de Monique ayant participé au marriage. Rédige les souvenirs à la première personne,
        Chaque souvenir doit être spécifique et détaillées et en intégrant les émotions ressenties."""),
    ]
)
chain = prompt_template | model.with_structured_output(LifeSummary)

In [11]:
life_summary = chain.invoke({"name": "Monique", "age": "75", "occupation": "Ancienne Assistante sociale"})

In [12]:
life_summary.model_dump()

{'name': 'Monique',
 'age': 75,
 'occupation': 'Ancienne Assistante sociale',
 'memories': [{'person': {'id': 1,
    'name': 'Marie',
    'relationship': "Amie d'enfance",
    'age': 74,
    'occupation': 'Retraitée',
    'hobbies': ['Jardinage', 'Lecture'],
    'description': "Marie est mon amie d'enfance, nous avons grandi ensemble et partagé de nombreux moments de vie."},
   'place': ['Église du village', 'Salle de réception'],
   'emotion': ['Joie', 'Nostalgie'],
   'description': "Le mariage de Thibault était magnifique. Voir Monique si heureuse m'a rappelé tous les moments que nous avons partagés. L'église du village était décorée avec goût, et la cérémonie était très émouvante. Pendant la réception, nous avons dansé et ri comme si nous étions encore jeunes. C'était un jour inoubliable."},
  {'person': {'id': 2,
    'name': 'Jean',
    'relationship': 'Frère',
    'age': 72,
    'occupation': 'Retraité',
    'hobbies': ['Pêche', 'Randonnée'],
    'description': 'Jean est le frère

In [14]:
with open("life_summary.json", "w", encoding="utf-8") as file:
    file.write(life_summary.model_dump_json(indent=2))

# Synthèse de l'histoire

In [15]:
with open("life_summary.json", "r", encoding="utf-8") as file:
    life_summary = json.load(file)

In [16]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage("""Tu est un assistant aidant les personnes agées atteintes de déficience de la mémoire à long terme
        à se rappeler des moments marquants de leur vie"""),
        ("user", """
         Voici ldes informations sur la personne agée :
            Nom : {name}
            Age : {age}
            Profession : {occupation}
         Voici différents souvenirs exprimées par des proches de Monique ayant participé au marriage de Thibault son petit fils :
            {memories}
         Raconte ce mariage à Monique sous une forme narative de 100 mots environ.
         Fais le avec l'intention que Monique puisse se souvenir de ces souvenirs.
         Adresse toi à Monique sur un ton amical et bienveillant.
         """),
    ]
)
chain = prompt_template | model

In [17]:
resume = chain.invoke({"name": "Monique", "age": "75", "occupation": "Ancienne Assistante sociale", "memories":life_summary})

In [ ]:
display(Markdown(resume.content))

Monique, tu te souviens du mariage de ton petit-fils Thibault ? C'était une journée magnifique, remplie de joie et d'émotion. La cérémonie à l'église du village était si émouvante, avec la décoration élégante et les moments de partage. Ensuite, à la salle de réception, tout le monde a dansé et ri, comme si nous étions encore jeunes. Ton amie d'enfance Marie était là, et vous avez partagé tant de souvenirs ensemble. Ton frère Jean était très fier de voir la famille réunie. Ta fille Sophie était remplie de bonheur de te voir si heureuse. C'était vraiment un jour inoubliable, entourée de tous tes proches.